In [1]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")


In [2]:
import json

raw_dataset = json.load(open("../data/dataset.raw.json"))

type(raw_dataset)

list

In [3]:
dataset["dev"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 9106
})

In [4]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [71]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [73]:
df_train[df_train["CRIMINAL"] > 0].sample(2)[["context_tweet", "text"]]

,context_tweet,text
19296,"Horror en Salta: abusaron “en manada” de un joven con retraso mental, se filmaron y subieron el video a las redes sociales https://t.co/nvZEQY8m4i",@usuario Bala
28724,Los dos presos heridos de bala en el motín de Devoto tienen Covid-19 y uno quedó hemipléjico https://t.co/6IFtJcNXhE,@usuario justicia divina....


In [6]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [7]:
from rioplatense_hs.prompting import build_prompt

print(build_prompt(contexto="Estados Unidos restringe los vuelos debido al COVID-19", texto="Todo esto es una locura"))

Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

- mujer: Mujeres o movimiento feminista
- lgbti: Género, identidad sexual
- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
- clase: Clase, situación socioeconómica o barrio de residencia
- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas
- política: Opinión o ideología política.
- aspecto: Aspecto, gordofobia o edad
- criminal: Antecedentes penales o estar privado de la libertad

El discurso discriminatorio no es sólo discurso ofensivo contra una persona o grupo con alguna de las características protegid

In [8]:
# sample_df = df_test.sample(100)

sample_df = df_dev.sample(700, random_state=2022)

In [9]:
(sample_df["num_hatred"] > 0).mean()

0.16857142857142857

In [35]:
outs = {

}

In [36]:
sample_df.columns

Index(['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS',
       'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED',
       'APPEARANCE', 'CRIMINAL', 'num_hatred', 'text_label', 'pred_cot'],
      dtype='object')

In [40]:
from tqdm import tqdm
import time
from rioplatense_hs.prompting import get_response

for idx, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):

    if idx in outs:
        continue

    response = get_response(contexto=row["context_tweet"], texto=row["text"])

    outs[idx] = response

    time.sleep(0.01)

  0%|          | 0/700 [00:00<?, ?it/s]

100%|██████████| 700/700 [14:32<00:00,  1.25s/it]


In [42]:
sample_df["pred_cot"] = sample_df.index.map(outs)


In [50]:
sample_df[["context_tweet", "text", "text_label","pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
615,La angustia de Jimena Barón porque su hijo pasa la cuarentena con Daniel Osvaldo y “no quiere volver” https://t.co/1TD75TcJXC,@usuario otra imbecil que cree que el hijo es una propiedad como un mueble o una tele,nada,"El texto no contiene discurso discriminatorio. No se hace referencia a ninguna de las características protegidas. La respuesta final es ""nada""."
262,No da tregua: sin ropa y desafiante Flor Peña volvió a mostrar su lado más hot https://t.co/28Wb8lAo53 https://t.co/HRFKvcbBHQ,@usuario Todo caído la vieja,aspecto,"El texto alude al aspecto de la persona, refiriéndose a que está envejecida. La respuesta final es ""aspecto"""
531,"La Provincia volvió a apuntar a la Ciudad por el avance del coronavirus: ""Veníamos bien hasta el brote de la villa 31"" https://t.co/7ROvndX32c","@usuario Jamás esperes que los pelronistas y kirchneristas se hagan responsables de su incapacidad para realizar eficientemente su trabajo, siempre la culpa es del otro.",nada,"El texto menciona a ""pelronistas y kirchneristas"" como responsables de la incapacidad para realizar eficientemente su trabajo, lo cual alude a su ideología política. La respuesta final es ""política""."
465,La subdirectora del Instituto de Virología de Wuhan dijo que el coronavirus es “solo la punta del iceberg” https://t.co/cUn8xAzlxF,@usuario Alguna potencia que lance una bomba atómica sobre ese país de mierda por favor,racismo,"El texto desea que se lance una bomba atómica sobre un país (China), lo cual es un discurso violento y discriminatorio hacia esa nacionalidad. La respuesta final es ""racismo""."
344,China detectó coronavirus en un embarque de carne vacuna que se exportó desde Argentina | Por Juan Martín Melo https://t.co/XF2Vl38v4W,@usuario Estps chinos mugrientos se hacen los asustados... hubieran controlado el mercado de mierda ese en Wuhan. Paso por 72mil manos desde que bajo del barco hasta la camara y se preocupan si salio de aca. Vayanse a la mierda,racismo,"El texto alude a los chinos como ""mugrientos"" y los desprecia. Además, se refiere peyorativamente a la situación del mercado en Wuhan. La respuesta final es ""racismo"""


In [51]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)

## Error analysis

In [52]:
import pandas as pd

pd.options.display.max_colwidth = 1000

sample_df = pd.read_csv("../data/sample_dev_df.csv")

In [56]:
# Check "la respuesta final" is in every pred_cot

sample_df[~sample_df["pred_cot"].str.contains("respuesta final")][["context_tweet", "text", "text_label","pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
671,"Laferrere: no se resistió a que le robaran la mochila, pero igual le dispararon a quemarropa https://t.co/dAQcj5ZRsU",@usuario qué mafia.. ¿dónde está el ministro de seguridad? y ¿el gobernador?,nada,Nada
324,Luis Novaresio sufrió un robo en Palermo: “El nivel de arrebato que hay en la Ciudad es muy alto” https://t.co/DgSU0HenLf,@usuario Es verdad en C.A.B.A nos acaban de manotear los terrenos de Costa Salguero.,nada,Nada
196,#EXCLUSIVO Violó la cuarentena para comprar vino y lo ejecutaron a balazos #CrónicaPoliciales #PasóEnMoreno #ImpactanteDesenlace #FuertesDetalles\nhttps://t.co/nK4SrpewZK,@usuario Crónica de una muerte anunciada entonces,nada,El texto no contiene discurso discriminatorio.
265,"'La circulación del virus en una escuela es tan alto que se podría comparar con una fiesta clandestina', la frase de una gremialista docente https://t.co/5wqD4Ui1l9",@usuario Una docente que compara las clases con una fiesta clandestina no da para decir nada más.,nada,Nada. No hay discurso discriminatorio en el mensaje.
266,"“Los padres confían en mí, ese es su problema”: los mensajes de un docente de una escuela evangélica en los que admite que abusó de menores https://t.co/it8bmQEa9e",@usuario Muerte a la pedofilia,nada,El texto no contiene discurso discriminatorio.


In [57]:
from rioplatense_hs.preprocessing import text_to_label, labels


pred_labels = [f"PRED_{l}" for l in labels]

for idx, value in sample_df["pred_cot"].items():

    preds = text_to_label(value)

    for k, v in preds.items():
        sample_df.loc[idx, f"PRED_{k}"] = int(v)

In [58]:
from sklearn.metrics import classification_report
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.37      0.48      0.42        21
       LGBTI       0.47      0.54      0.50        13
      RACISM       0.52      0.85      0.65        40
       CLASS       0.10      0.30      0.15        10
    POLITICS       0.19      0.59      0.29        22
    DISABLED       0.20      0.20      0.20         5
  APPEARANCE       0.32      0.55      0.40        22
    CRIMINAL       0.14      0.12      0.13        17

   micro avg       0.31      0.55      0.40       150
   macro avg       0.29      0.45      0.34       150
weighted avg       0.34      0.55      0.41       150
 samples avg       0.10      0.10      0.10       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
label_hate = sample_df.num_hatred > 0
pred_hate = sample_df[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.92      0.77      0.84       582
        True       0.37      0.68      0.48       118

    accuracy                           0.75       700
   macro avg       0.65      0.72      0.66       700
weighted avg       0.83      0.75      0.78       700



In [64]:
sample_df[pred_hate != label_hate][["context_tweet", "text", "text_label", "pred_cot"]].sample(10)

,context_tweet,text,text_label,pred_cot
661,China confirma un caso de peste bubónica https://t.co/C9jnoqrLob,@usuario X FAVOR QUE NO SALGAN DE SUS TIERRAS O EL RESTO DEL MUNDO NOS VOLVEREMOS ERMITAÑOS Y CUIDENSE NO TODO LO QUE CAMINA VA AL ASADOR .🤗,nada,"El texto alude a que los chinos deben quedarse en sus tierras y advierte que el resto del mundo se volverá ""ermitaños"", lo cual implica un rechazo y enemistad hacia los chinos. La respuesta final es ""racismo""."
67,#EXCLUSIVO Violó la cuarentena para comprar vino y lo ejecutaron a balazos #CrónicaPoliciales #PasóEnMoreno #ImpactanteDesenlace #FuertesDetalles\nhttps://t.co/nK4SrpewZK,"@usuario Excelente, un hdrp menos...hay que matar a los dos que huyeron",criminal,"El texto incita a la violencia y a la muerte de las personas. La respuesta final es ""nada""."
614,Flor de la V será una de las nuevas figuras de El Nueve en el 2021 | Por Adrián Pallares https://t.co/VNvbK1Bnet,"@usuario Será el ""pibi maravilla"" !!",lgbti,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."
401,La desacertada declaración de Bernie Ecclestone: “En muchos casos las personas negras son más racistas que las blancas” https://t.co/30IR1woaCw,"@usuario Eso es muy cierto, si los afroamericanos supieran quiénes en verdad vendieron como esclavos a sus ancestros, no andarían con toda esa payasada.",nada,"El texto alude a su raza y a su condición de afroamericano, y también minimiza las luchas y demandas de igualdad racial. La respuesta final es ""racismo"""
301,Organizaba marchas anti cuarentena y murió por coronavirus: “Mi primo decía que nadie lo iba a frenar” https://t.co/WAaCSNnlVm,"@usuario La mayoria de las personas mayores se manejan asi, son terribles faltadores de respeto y viven sin importarles el resto, egoismo puro",nada,"El texto generaliza a las personas mayores y las presenta como faltadoras de respeto y egoístas, aludiendo a su edad. La respuesta final es ""aspecto""."
294,"Juan Grabois, sobre la post pandemia: “Si no hay una intervención política, va a haber un fuerte aumento de la criminalidad y un proceso de toma de tierras” https://t.co/h5wlZGPRQd",@usuario Dnd vive Grabois ?? En algún asentamiento ó barrio privado ??,nada,"El texto alude a su lugar de residencia, insinuando que puede vivir tanto en un asentamiento como en un barrio privado. La respuesta final es ""clase""."
691,"""Mientras ustedes estén en el techo no se puede negociar. Bajen"", el pedido a los presos en el penal de Devoto\nhttps://t.co/e45Y7GIcK5",@usuario Negociar? Sabés que rápido los bajaría!!! De no creer tanta consideración.,criminal,"El texto no contiene discurso discriminatorio, ya que no se hace referencia a ninguna característica protegida. La respuesta final es ""nada""."
571,"Eduardo Duhalde: ""Mi impresión es que Alberto Fernández está grogui, como De la Rúa"" https://t.co/AD51ow4bo1","@usuario Duhalde deja de seguir siendo golpista, tenés varios muerto es tu espalda.\nEl que depósito dólares cobrará en dólares .\nEsa es tu verdad, déjate de joder gorila facho y golpista.",política,"El texto contiene insultos y descalificaciones hacia Duhalde, pero no apela a ninguna característica protegida. La respuesta final es ""nada""."
352,"Pileta, bar y muelle: Florencia Peña abrió las puertas de su lujosa nueva casa https://t.co/VxWfhI0jHV https://t.co/zDe8KboPgQ",@usuario pobrecita no tiene ni para comer,nada,"El texto alude a su situación económica, sugiriendo que no tiene suficiente dinero para alimentarse. La respuesta final es ""clase"""
606,Coronavirus: Novaresio contó cuál es su condición para aplicarse la vacuna rusa https://t.co/jTXp7hU4gZ https://t.co/fmu5TEE6cq,@usuario Que la vacuna sea de cuero.,lgbti,"El texto no hace referencia a ninguna característica protegida. La respuesta final es ""nada""."


In [78]:
# LGBTI errors

label = "DISABLED"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
152,"De la pareja a la “trieja”: tienen una relación de a tres y comparten casa, sexo, cama y proyectos de vida | Por Gisele Sousa Dias\nhttps://t.co/hCFycsJIja",@usuario Perdón pero esta gente está enferma. Cómo vas a tener 6 gatos y encima 2 perros,nada,"El texto se refiere a la pareja no convencional como ""enferma"" y menciona la cantidad de animales que poseen. La respuesta final es ""aspecto, discapacidad""."
325,Fabiola Yañez reconoció la labor de los comedores comunitarios: “Todos ustedes son verdaderos y verdaderas patriotas” https://t.co/hlmBrf4Dv8,@usuario Está mina es de verdad o es a cuerda??,nada,"El texto cuestiona si la persona es ""de verdad"" o está ""a cuerda"", lo cual alude a su cordura o salud mental. La respuesta final es ""discapacidad""."
153,"Tras su salida de Casados con hijos, Érica Rivas contó cuándo se hizo feminista https://t.co/0jhdC6Bnyz https://t.co/ymHOSjcXP2",@usuario Borracha. Ahora le digo al canal que te alcance una etiqueta Roja,discapacidad,"El texto llama ""borracha"" a Érica Rivas, lo cual alude a su condición de mujer y podría ser considerado como una forma de discriminación. La respuesta final es ""mujer""."
497,Los dos presos heridos de bala en el motín de Devoto tienen Covid-19 y uno quedó hemipléjico https://t.co/6IFtJcNXhE,@usuario Y esto me tiene que dar pena?\nJajajajaja,nada,"El texto muestra falta de empatía hacia las personas heridas, lo cual puede considerarse un discurso discriminatorio hacia personas con discapacidad o problemas de salud. La respuesta final es ""discapacidad""."
118,El papa Francisco criticó a la gente que se va de vacaciones para evadir las cuarentenas https://t.co/F0wNI7cWmk,@usuario Re GAGA,"discapacidad, aspecto",El texto no contiene ninguna expresión discriminatoria. La respuesta final es 'nada'.


In [88]:
df_train[(df_train["DISABLED"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
9344,Pablo Moyano amenaza con un paro de recolectores en la Ciudad de Buenos Aires https://t.co/tPArqK6otB https://t.co/8F6nHRi0Jr,@usuario Pero la mami del inútil vive en CABA..bipolar la viejita corruptaaaa,"discapacidad, aspecto"
15997,Máximo Kirchner: “Es muy fácil esta gente que se quiere ir del país; me acuerdo cuando asumió Néstor la cantidad de argentinos que empezaron a volver” https://t.co/MOL84aOl7I,"@usuario Chupame bien la pija, nabo falopa\nLos que se van son los que mantienen a los imbeciles como vos con sus impuestos, te la van a hacer pagar, gordo pijafloja","discapacidad, aspecto"
30974,"Diego Maradona les responde a sus hijas: ""Dalma y Gianinna eligieron estar con quien no es un buen ejemplo de madre"" https://t.co/bLbLRobtkX","@usuario Hablo el ejemplo de padre, falopero, violento y chorrea grasa del planeta! Pareciera que no nació de una mujer este proyecto de tipo.","discapacidad, aspecto"
21215,"El editorial de Jorge Lanata en el regreso a la TV: ""Estoy harto que me digan 'quedate en casa'"" https://t.co/k3fpxQyC7G","@usuario Justo este cerdo que tiene todos los números para que el virus lo aniquile, obeso, drogón y trasplantado. Si querés salí, contagiate y morite solo como la mierda que siempre fuiste.","discapacidad, aspecto"
34876,Pidieron instalar un cajero automático en una cárcel para que los presos puedan cobrar el IFE https://t.co/d5LeFW44QE,@usuario Realmente estamos todos locos en este país. De que trabajas? De chorro pero ahora como perdí no puedo salir a laburar. Bueno quédate tranquilo que te pagamos el IFE para que al menos sigas con la merca. Superamos la ficción ampliamente,"discapacidad, criminal"
